In [1]:
from api_key import gkey


In [2]:
%load_ext nb_black
import warnings
warnings.filterwarnings("ignore")
# ! jupyter nbextension enable --py --sys-prefix widgetsnbextension
# ! jupyter nbextension enable --py --sys-prefix gmaps


<IPython.core.display.Javascript object>

In [3]:
# !pip install gmaps
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

file = "../output_data/cities.csv"
df = pd.read_csv(file)


<IPython.core.display.Javascript object>

In [26]:
ideal_location = df.loc[(df["Max Temp"] < 77) & (df["Max Temp"] > 75), :]
ideal_location = df.loc[(df["Cloudiness"] == 0) & (df[" Wind Speed"] < 1), :]
ideal_location.dropna()
ideal_location = ideal_location.reset_index(drop=True)
ideal_location.count()

City           3
Lat            3
Lng            3
Max Temp       3
Humidity       3
Cloudiness     3
 Wind Speed    3
Country        3
Date           3
dtype: int64

<IPython.core.display.Javascript object>

In [27]:
ideal_location.head(2)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Karachi,24.9056,67.0822,26.90,29,0,0.0,PK,1645080743
1,North Bend,43.4065,-124.2243,6.07,99,0,0.0,US,1645080802


<IPython.core.display.Javascript object>

In [32]:
hotel_df = pd.DataFrame()
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "",
    "radius": "50000",
    "keyword": "hotel",
    "key": gkey,
}
lat = [lat for lat in ideal_location["Lat"]]
lng = [lng for lng in ideal_location["Lng"]]

for index, row in ideal_location.iterrows():

    get_location = str(lat[index]) + "," + str(lng[index])
    params["location"] = get_location
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        hotel_df.loc[index, "City"] = row["City"]
        hotel_df.loc[index, "Country"] = row["Country"]
        hotel_df.loc[index, "Lat"] = results[0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Lng"] = results[0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        continue

<IPython.core.display.Javascript object>

In [33]:
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,Beach Luxury Hotel,Karachi,PK,24.843392,66.995550
1,Red Lion Hotel Coos Bay,North Bend,US,43.377778,-124.216944
2,Hospitality Palace Hotel,Kifrī,IQ,34.932714,44.595047


<IPython.core.display.Javascript object>

In [29]:
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params = {
#     "location": "39.952583,-75.16522",
#     "radius": "50000",
#     "keyword": "hotel",
#     "key": gkey,
# }
# response = requests.get(base_url, params=params).json()
# results = response["results"]
# results[0]["geometry"]["location"]

<IPython.core.display.Javascript object>

In [52]:
gmaps.configure(gkey)
intensity = df["Humidity"].max()
fig = gmaps.figure(zoom_level=2, center=(41.4036007, -95.0138776),)
heatmap = gmaps.heatmap_layer(df[["Lat", "Lng"]], weights=df["Humidity"],)
heatmap.max_intensity = intensity
heatmap.point_radius = 15
fig.add_layer(heatmap)
fig


Figure(layout=FigureLayout(height='420px'))

<IPython.core.display.Javascript object>

In [53]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
ideal_location2 = hotel_df[["Lat", "Lng"]]
layers = gmaps.marker_layer(ideal_location2,info_box_content=hotel_info)
fig.add_layer(layers)
fig


Figure(layout=FigureLayout(height='420px'))

<IPython.core.display.Javascript object>